In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
economy = pd.read_csv('economy_related_rank.csv')
economy.head()

,City,State,Percent unemployment,State sale tax rate,Local tax rate,Total rate,Median Income,AvgTuition,Rank_Unemployment,Rank_Sales,Rank_Income,Rank_Tuition,Sum,Economy_rank,Economy_ref,Lat,Lon
0,Atlanta,GA,10.2,0.040,0.049,0.089,59345.0,10309.0,30.0,44.5,22,15.0,111.5,37.0,14.0,33.749099,-84.390185
1,Austin,TX,6.3,0.063,0.020,0.083,65886.0,8729.0,4.0,27.5,29,21.5,82.0,17.0,34.0,30.271129,-97.743700
2,Baltimore,MD,11.1,0.060,0.000,0.060,50492.0,12273.0,34.5,4.5,10,10.0,59.0,4.0,47.0,39.290882,-76.610759
3,Birmingham,AL,NaN,4.050,0.060,0.100,NaN,NaN,35.5,47.0,36,38.0,156.5,47.0,4.0,33.520682,-86.802433
4,Boston,MA,6.6,0.063,0.000,0.063,62180.0,20442.0,6.0,4.5,25,1.0,36.5,1.0,50.0,42.360482,-71.059568


In [ ]:
app = dash.Dash()

available_indicators = ['Rank_Unemployment', 'Rank_Sales','Rank_Income', 'Rank_Tuition']
app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label('First care'),
            dcc.Dropdown(
                id='First-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'crime_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        
        html.Div([
            html.Label('Second care'),
            dcc.Dropdown(
                id='Second-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        
        html.Div([
            html.Label('Third care'),
            dcc.Dropdown(
                id='Third-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
       
        html.Div([
            html.Label('Fourth care'),
            dcc.Dropdown(
                id='Fourth-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        html.Button('Submit', id='button'),
        
        html.Div(dcc.Graph(id='indicator-graphic'))
    ]),
    #dcc.Graph(id='indicator-graphic')
])

@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value'),
     dash.dependencies.State('Third-care', 'value'),
     dash.dependencies.State('Fourth-care', 'value')])
    
def update_graph(Button, First_care, Second_care):
    df = pd.DataFrame()
    df['City'] = economy['City']
    df['First'] = economy[First_care]
    df['Second'] = economy[Second_care]
    df['Third'] = economy[Third_care]
    df['Fourth'] =  economy[Fourth_care]
    df['Total'] = (df['First']+df['Second']+df['Third']+df['Fourth']).rank(ascending=1)
    df['reverse_rank'] = df['Total'].rank(ascending=0)
    df['text'] = df['City'] + '<br># Final Rank ' + (df['Total']).astype(str)
    df['longitude'] = economy['Lon']
    df['latitude'] = economy['Lat']
    
    limits = [(0,10),(10,20),(20,30),(30,40),(40,50)]
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
    cities = []


    for i in range(len(limits)):
        lim = limits[i]
        df_sub = df[lim[0]:lim[1]]
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['reverse_rank']*15,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0],lim[1]) )
        cities.append(city)

        layout = dict(
            title = 'The economy related ranking of US big cities',
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )
        fig = dict( data=cities, layout=layout )
        
    return {   
        plotly.offline.plot( fig, validate=False )     
    }

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2017 12:34:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2017 12:34:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2017 12:34:32] "GET /_dash-dependencies HTTP/1.1" 200 -
[2017-11-22 12:34:32,760] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "D:\Anoconda\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anoconda\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anoconda\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anoconda\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "D:\Anoconda\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anoconda\li